In [1]:
import gcsfs
import zarr
import dask
import xarray as xr
from dask.diagnostics import ProgressBar
from distributed import Client,LocalCluster


In [2]:
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 24,Total memory: 63.25 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51337,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 63.25 GiB
Comm: tcp://127.0.0.1:51371,Total threads: 4
Dashboard: http://127.0.0.1:51372/status,Memory: 10.54 GiB
Nanny: tcp://127.0.0.1:51340,


In [4]:
ds = xr.open_zarr(
    's3://ncar-cesm2-lens/ocn/monthly/cesm2LE-ssp370-cmip6-TEMP.zarr',
    storage_options={'anon': True},
)

In [5]:
ds

<xarray.Dataset> Size: 2TB
Dimensions:     (member_id: 50, time: 1032, z_t: 60, nlat: 384, nlon: 320, d2: 2)
Coordinates:
  * member_id   (member_id) <U12 2kB 'r10i1181p1f1' ... 'r9i1301p1f1'
  * time        (time) object 8kB 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
    time_bound  (time, d2) object 17kB dask.array<chunksize=(1032, 2), meta=np.ndarray>
  * z_t         (z_t) float32 240B 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
Dimensions without coordinates: nlat, nlon, d2
Data variables:
    TEMP        (member_id, time, z_t, nlat, nlon) float32 2TB dask.array<chunksize=(1, 6, 60, 384, 320), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    calendar:          All years have exactly  365 days.
    cell_methods:      cell_methods = time: mean ==> the variable values are ...
    contents:          Diagnostic and Prognostic Variables
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    revision:          $Id$
    source:            CCSM POP2, the CCSM Ocean Component
    time_period_freq:  month_1

In [6]:
ds_first_five_years = ds.sel(time=slice('2015', '2019')).isel(member_id=range(5))
ds_first_five_years

<xarray.Dataset> Size: 9GB
Dimensions:     (member_id: 5, time: 60, z_t: 60, nlat: 384, nlon: 320, d2: 2)
Coordinates:
  * member_id   (member_id) <U12 240B 'r10i1181p1f1' ... 'r10i1301p1f1'
  * time        (time) object 480B 2015-01-16 12:00:00 ... 2019-12-16 12:00:00
    time_bound  (time, d2) object 960B dask.array<chunksize=(60, 2), meta=np.ndarray>
  * z_t         (z_t) float32 240B 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
Dimensions without coordinates: nlat, nlon, d2
Data variables:
    TEMP        (member_id, time, z_t, nlat, nlon) float32 9GB dask.array<chunksize=(1, 6, 60, 384, 320), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    calendar:          All years have exactly  365 days.
    cell_methods:      cell_methods = time: mean ==> the variable values are ...
    contents:          Diagnostic and Prognostic Variables
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    revision:          $Id$
    source:            CCSM POP2, the CCSM Ocean Component
    time_period_freq:  month_1

In [7]:
resampled = ds_first_five_years.resample(time='AS').mean('time')


<string>:6: FutureWarning: 'AS' is deprecated and will be removed in a future version. Please use 'YS' instead of 'AS'.


In [8]:
resampled

<xarray.Dataset> Size: 737MB
Dimensions:    (time: 5, member_id: 5, z_t: 60, nlat: 384, nlon: 320)
Coordinates:
  * member_id  (member_id) <U12 240B 'r10i1181p1f1' ... 'r10i1301p1f1'
  * z_t        (z_t) float32 240B 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
  * time       (time) object 40B 2015-01-01 00:00:00 ... 2019-01-01 00:00:00
Dimensions without coordinates: nlat, nlon
Data variables:
    TEMP       (time, member_id, z_t, nlat, nlon) float32 737MB dask.array<chunksize=(1, 1, 60, 384, 320), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    calendar:          All years have exactly  365 days.
    cell_methods:      cell_methods = time: mean ==> the variable values are ...
    contents:          Diagnostic and Prognostic Variables
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    revision:          $Id$
    source:            CCSM POP2, the CCSM Ocean Component
    time_period_freq:  month_1

In [9]:
%%time
yearly_average_weighted_incorrectly = resampled.compute()

CPU times: total: 1.52 s
Wall time: 50.8 s
